# Tool Use Pattern - Agentic AI Design Patterns 01

## I. Mô hình Tool Use (Kết hợp công cụ)
### 1. Overview
- *Là cơ chế quan trọng cho phép mô hình AI tương tác với các hệ thống bên ngoài*, APIs, hoặc tài nguyên vượt qua khả năng bên trong của chính mô hình (tương tác thời gian thực, các công cụ đặc biệt).
- Giải quyết hạn chế truyền thống của LLMs - thường bị giới hạn bởi dữ liệu huấn luyện.
- Sử dụng kiến trúc mô-đun hóa, trong đó các nhiệm vụ được gán cho các công cụ chuyên dụng, nâng cao hiệu quả, tính linh hoạt và khả năng mở rộng.
- Agentic AI có thể tự động lựa chọn, sử dụng và phối hợp nhiều công cụ để thực hiện các nhiệm vụ phức tạp mà không cần sự can thiệp liên tục của con người, thể hiện khả năng giải quyết vấn đề tiên tiến
- LLàm nổi bật các tính năng chủ động chính như ra quyết định, khả năng thích ứng và khả năng học hỏi từ việc sử dụng công cụ, mở đường cho các hệ thống AI tự chủ và đa năng hơn.

### 2. Tool Use Pattern in Agentic AI: An Overview
- **Tool Use**: công nghệ mạnh mẽ, đại diện cho một khuôn mẫu thiết kế thay đổi cách mà LLMs thực thi.
- Công nghệ cách mạng này cho phép LLMs vượt qua giới hạn tự nhiên bằng việc tương tác với các chức năng bên ngoài để thu thập thông tin, thực hiện hành động, và thao tác dữ liệu.

- Giới hạn tự nhiên của LLMs:
	- Sự phát triển ban đầu của LLMs tập trung vào việc sử dụng các Transformer tiền huấn luyện để dự đoán token tiếp theo trong chuỗi.
	- Tuy đây là một bước cách mạng, nhưng khả năng LLMs bị giới hạn ở dữ liệu huấn luyện.

### 3. Architecture

<p align="center"><img src="./attachments/ToolUsePipe.png" alt="" width="700"/></p>

- Sơ đồ trên thể hiện một mô hình Tool Use, trong đó một hệ thống AI tương tác với nhiều công cụ chuyên dụng để xử lý hiệu quả các truy vấn của người dùng bằng cách truy cập các nguồn thông tin khác nhau.
- Cách tiếp cận này là một phần của phương pháp luận đang phát triển trong AI được gọi là Agentic AI, được thiết kế để nâng cao khả năng xử lý các nhiệm vụ phức tạp của AI bằng cách tận dụng các tài nguyên bên ngoài.

- **Ý tưởng chính**;
	- Mô-đun hóa các nhiệm vụ:
		- Hệ thống chia nhỏ nhiệm vụ - gán nó cho các công cụ khác nhau;
		- Đảm bảo tổng hợp hiệu quả và tính mở rộng.
	- Sử dụng các công cụ chuyên dụng cho từng nhiệm vụ riêng biệt.
	- Xử lý tuần tự:
		- Chạy các truy vấn tuần tự thông qua các công cụ.

### 4. How does an LLM Identify the Need for a Tool?

<p align="center"><img src="./attachments/Tools.png" alt="" width="500"/></p>

- **Quy trình End-to-End**:
	1. Input: yêu cầu người dùng (ví dụ: "What is 2 times 3?")
	2. Interpretation: Xác định nhiệm vụ (Đây là tác vụ tính toán)
	3. Tool Selection: Lựa chọn công cụ (Công cụ nhân)
	4. Payload Creation: Xác định các biến liên quan, chuẩn bị và thực thi trên các công cụ.
	5. Execution: Công cụ thực hiện nhiệm vụ và trả kết quả về cho LLMs.

## II. Cài đặt minh họa

**Setup**

In [1]:
import os
import logging
from langchain import PromptTemplate, LLMChain
from langchain_core.runnables import RunnableSequence
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import Tool, initialize_agent
from IPython.display import Markdown, display

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
os.environ["GOOGLE_API_KEY"] = input("Your Gemini API key: ")

#GeminiAPI
llm = ChatGoogleGenerativeAI(
    model       = "gemini-2.0-flash",
    temperature = 0.55,
    max_tokens  = None,
    timeout     = None,
    max_retries = 2,
)

**Tool**

In [3]:
def calculator(input: str) -> str:
    try:
        allowed = "0123456789+-*/.() "
        if not all(char in allowed for char in input):
            return "Error"
        
        result = eval(input)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

In [4]:
#Package Tool -> Langchain's Tool
calc_tool = Tool(
    name = "Calculator",
    func = calculator,
    description = (
        "Tool thực hiện các phép tính số học cơ bản (cộng, trừ, nhân, chia). "
        "Chỉ nhận các biểu thức dạng '2+2', '3 * 4', '10/2', v.v."
    )
)

tools = [calc_tool]

**Agent**

In [5]:
agent = initialize_agent(
    tools,
    llm,
    agent   = "zero-shot-react-description",
    verbose = True
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22356\2981477701.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


**Test**

In [6]:
user_input = input("Nhập biểu thức số học (ví dụ: 3+5*2): ")

result = agent.run(user_input)
print("Kết quả:", result)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22356\3746904879.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(user_input)




> Entering new AgentExecutor chain...
I need to evaluate the expression 5+6*9/(2+1). I should follow the order of operations (PEMDAS/BODMAS). First, I evaluate the expression inside the parentheses.
Action: Calculator
Action Input: 2+1
Observation: 3
Thought:Now I need to evaluate 5+6*9/3. Next, I perform the multiplication and division from left to right. So, I first calculate 6*9.
Action: Calculator
Action Input: 6*9
Observation: 54
Thought:Now I need to evaluate 5+54/3. Next, I perform the division. So, I calculate 54/3.
Action: Calculator
Action Input: 54/3
Observation: 18.0
Thought:Now I need to evaluate 5+18. Finally, I perform the addition.
Action: Calculator
Action Input: 5+18
Observation: 23
Thought:I now know the final answer
Final Answer: 23

> Finished chain.
Kết quả: 23
